In [20]:
import numpy as np
import torch
from smplx import SMPLX
from mocap_dataset_process.utils.face_z_align_util import joint_idx, face_z_transform
from mocap_dataset_process.utils.rotation_conversions import *
import os


def compute_canonical_transform(global_orient):
    rotation_matrix = torch.tensor([
        [1, 0, 0],
        [0, 0, 1],
        [0, -1, 0]
    ], dtype=global_orient.dtype)
    global_orient_matrix = axis_angle_to_matrix(global_orient)
    global_orient_matrix = torch.matmul(rotation_matrix, global_orient_matrix)
    global_orient = matrix_to_axis_angle(global_orient_matrix)
    return global_orient

def transform_translation(trans):
    trans_matrix = np.array([[1.0, 0.0, 0.0], [0.0, 0.0, 1.0], [0.0, 1.0, 0.0]])
    trans = np.dot(trans, trans_matrix)  # exchange the y and z axis
    trans[:, 2] = trans[:, 2] * (-1)
    return trans


def get_smplx_322(data, ex_fps):
    fps = 0


    if 'mocap_frame_rate' in data:
        fps = data['mocap_frame_rate']
        print(fps)
        down_sample = int(fps / ex_fps)
        
    elif 'mocap_framerate' in data:
        fps = data['mocap_framerate']
        print(fps)
        down_sample = int(fps / ex_fps)
    else:
        # down_sample = 1
        return None

    frame_number = data['trans'].shape[0]
    


    fId = 0 # frame id of the mocap sequence
    pose_seq = []

    

    for fId in range(0, frame_number, down_sample):
        pose_root = data['root_orient'][fId:fId+1]
        pose_body = data['pose_body'][fId:fId+1]
        pose_hand = data['pose_hand'][fId:fId+1]
        pose_jaw = data['pose_jaw'][fId:fId+1]
        pose_expression = np.zeros((1, 50))
        pose_face_shape = np.zeros((1, 100))
        pose_trans = data['trans'][fId:fId+1]
        pose_body_shape = data['betas'][:10][None, :]
        pose = np.concatenate((pose_root, pose_body, pose_hand, pose_jaw, pose_expression, pose_face_shape, pose_trans, pose_body_shape), axis=1)
        pose_seq.append(pose)

    pose_seq = np.concatenate(pose_seq, axis=0)
    

    return pose_seq

def process_pose(pose):
    pose_root = pose[:, :3]
    pose_root = compute_canonical_transform(torch.from_numpy(pose_root)).detach().cpu().numpy()
    pose[:, :3] = pose_root
    pose_trans = pose[:, 309:312]
    pose_trans = transform_translation(pose_trans)
    pose[:, 309:312] = pose_trans

    return pose


def face_z_align(pose, smplx_model):
    pose = torch.from_numpy(pose).float().cuda()

    param = {
        'root_orient': pose[:, :3],  # controls the global root orientation
        'pose_body': pose[:, 3:3+63],  # controls the body
        'pose_hand': pose[:, 66:66+90],  # controls the finger articulation
        'pose_jaw': pose[:, 66+90:66+93],  # controls the yaw pose
        'face_expr': pose[:, 159:159+50],  # controls the face expression
        'face_shape': pose[:, 209:209+100],  # controls the face shape
        'trans': pose[:, 309:309+3],  # controls the global body position
        'betas': pose[:, 312:],  # controls the body shape. Body shape is static
    }

    batch_size = param['face_expr'].shape[0]
    zero_pose = torch.zeros((batch_size, 3)).float().cuda()

    smplx_output = smplx_model(betas=param['betas'], body_pose=param['pose_body'],
                               global_orient=param['root_orient'], pose2rot=True, jaw_pose=zero_pose, leye_pose=zero_pose, reye_pose=zero_pose,
                               left_hand_pose=param['pose_hand'][:, :45], right_hand_pose=param['pose_hand'][:, 45:],
                               expression=param['face_expr'][:, :10], transl=param['trans']) # jaron: 这里加载smplx_model的目的是得到joints，然后对它选择+位移，使得角色是面朝z轴（处于原点）
    print(smplx_output)          
    vertices = smplx_output.vertices
    joints = smplx_output.joints
    joints = joints[:, joint_idx, :]
    param['root_orient'], param['trans'] = face_z_transform(joints.cpu().numpy(), param['root_orient'], param['trans'])

    pose_list = []
    for k in ['root_orient', 'pose_body', 'pose_hand', 'pose_jaw', 'face_expr', 'face_shape', 'trans', 'betas']:
        pose_list.append(param[k])
    pose_list = torch.cat(pose_list, dim=-1).cpu().numpy()

    return pose_list


data = np.load(r"D:\jarondu\Datasets\motion_X_two\AMASS\BMLmovi\BMLmovi\Subject_1_F_MoSh\Subject_1_F_1_stageii.npz", allow_pickle=True)
ex_fps = 30
smplx_male_model_path = os.path.join("body_models/smplx", "SMPLX_MALE.npz")
smplx_female_model_path = os.path.join("body_models/smplx", "SMPLX_FEMALE.npz")

gender = data['gender'].item()

if gender == 'male':
    smplx_model = SMPLX(smplx_male_model_path, num_betas=10, use_pca=False, use_face_contour=True, batch_size=1).cuda()
elif gender == 'female':
    smplx_model = SMPLX(smplx_female_model_path, num_betas=10, use_pca=False, use_face_contour=True, batch_size=1).cuda()
    
pose = get_smplx_322(data, ex_fps)
pose = process_pose(pose)

pose = face_z_align(pose, smplx_model)




120.0


## smplx_blender_addon加载.npz文件，.npz中不能缺少的{key: value}
- 'gender'
- 'mocap_frame_rate'
- 'trans': (T,3)
- 'pose': (T,165)
- 'betas': 16

In [166]:
data = np.load(r"D:\jarondu\Datasets\motion_X_two\AMASS\BMLmovi\BMLmovi\Subject_1_F_MoSh\Subject_1_F_1_stageii.npz", allow_pickle=True)
print(dict(data).keys())
# print(dict(data))
# print(data['poses'].shape) 
# print(data['root_orient'].shape) # equal to: data['poses'][..., 0:3]
# print(data['pose_body'].shape) # equal to: data['poses'][..., 3:66]
# print(data['pose_jaw'].shape) # equal to: data['poses'][..., 66:69]
# print(data['pose_eye'].shape) # equal to: data['poses'][..., 69:75]
# print(data['pose_hand'].shape) # equal to: data['poses'][..., 75:165]


# joint_data = np.load(r"D:\jarondu\Notes\Text-to-Motion\Evaluation\【20240716】第一批打分\MoMask_Original\MoMask_Original\000000\repeat_0\joint\000000_repeat0_len196.npy")
# print(f"joint_data.shape{joint_data.shape}")
# joint_body = joint_data.reshape(-1,66)[...,3:]
# print(f"joint_body.shape:{joint_body.shape}")
# joint_orient = joint_data.reshape(-1,66)[...,:3]
# print(f"joint_body.shape:{joint_body.shape}")


droped_key = ['markers_latent_vids', 'markers_latent', 'latent_labels', 'markers_obs', 'markers_sim', 'marker_meta', 'num_markers', 'markers_latent_vids', 'markers', \
              'surface_model_type', 'mocap_time_length', 'labels', 'labels_obs', \
              'num_betas', 'root_orient', 'pose_body', 'pose_hand', 'pose_jaw', 'pose_eye']
modified_data = {}
for key in list(data.keys()):
    if key in droped_key:
        continue
    modified_data[key] = data[key]
np.savez('./test.npz',**modified_data)


dict_keys(['gender', 'surface_model_type', 'mocap_frame_rate', 'mocap_time_length', 'markers_latent', 'latent_labels', 'markers_latent_vids', 'trans', 'poses', 'betas', 'num_betas', 'root_orient', 'pose_body', 'pose_hand', 'pose_jaw', 'pose_eye', 'markers', 'labels', 'markers_obs', 'labels_obs', 'markers_sim', 'marker_meta', 'num_markers'])
(581, 3)


In [60]:
data1 = np.load('test.npz', allow_pickle=True)
print(dict(data1).keys())
print(type(data1['trans']))
print(data1['poses'].shape)
print(type(data1['betas']))

dict_keys(['trans', 'gender', 'mocap_frame_rate', 'mocap_framerate', 'betas', 'dmpls', 'poses'])
<class 'numpy.ndarray'>
(1955, 156)
<class 'numpy.ndarray'>


In [2]:
# import numpy as np
# data2 = np.load(r'C:\Users\jarondu\Downloads\ACCAD\ACCAD\s011\walkdog_poses.npz', allow_pickle=True)
# print(dict(data2).keys())
# print(data2['trans'].shape)
# print(data2['poses'].shape)
# print(data2['betas'].shape)
# print(data2['mocap_framerate'])
# modified_data = {}
# for key in list(data2.keys()):
#     if key == "mocap_framerate":
#         modified_data['mocap_frame_rate'] = data2[key]
#     modified_data[key] = data2[key]
# np.savez('./test.npz',**modified_data)

dict_keys(['trans', 'gender', 'mocap_framerate', 'betas', 'dmpls', 'poses'])
(1955, 3)
(1955, 156)
(16,)
120.0


In [7]:
data_smpl = np.load(r'./text_smpl.npz', allow_pickle=True)
print(dict(data_smpl).keys())

dict_keys(['vertices', 'joints', 'global_orient', 'betas', 'body_pose'])


In [85]:
# 创建一个smplx模型，然后往里面填参数
from smplx import SMPLX
import torch
import numpy as np
from scipy.spatial.transform import Rotation as R

def rotate(rotvec, rotation):
    '''
    input:
        rotvec: 轴角表示的旋转
        rotation: 需要旋转的旋转向量
    return:
        rotvec_res: 旋转后的轴角表示
    '''

    # 创建沿x轴旋转90度的Rotation对象
    x_rotation_90 = R.from_rotvec(rotation)

    # 将root_orient数组转换为Rotation对象数组
    root_orient_rotations = R.from_rotvec(root_orient)

    # 批量复合旋转
    final_rotations = x_rotation_90 * root_orient_rotations

    # 将复合后的旋转转换回轴角表示
    rotvec_res = final_rotations.as_rotvec()

    return rotvec_res

smplx_male_model_path = os.path.join('./body_models/smplx', "SMPLX_MALE.npz")
smplx_female_model_path = os.path.join('./body_models/smplx', "SMPLX_FEMALE.npz")
gender = "male"
if gender == 'male':
    smplx_model = SMPLX(smplx_male_model_path, num_betas=10, use_pca=False, use_face_contour=True, batch_size=1).cuda()
elif gender == 'female':
    smplx_model = SMPLX(smplx_female_model_path, num_betas=10, use_pca=False, use_face_contour=True, batch_size=1).cuda()


data_smpl = np.load(r'./text_smpl.npz', allow_pickle=True)
print(dict(data_smpl).keys())
print(data_smpl['global_orient'].shape)
print(data_smpl['betas'].shape)
print(data_smpl['body_pose'][...,:63].shape)

batch_size = data_smpl['body_pose'].shape[0]
root_orient = data_smpl['global_orient']
body_pose = data_smpl['body_pose'][...,:63]
jaw_pose = np.zeros((batch_size, 3))
eye_pose = np.zeros((batch_size, 6))
hand_pose = np.zeros((batch_size, 90))
print(f"hand_pose:{hand_pose.shape}")

# 需要施加的旋转向量
x_rotation_90_rad = np.deg2rad(90)
rotation = np.array([1, 0, 0]) * x_rotation_90_rad
root_orient = rotate(root_orient, rotation)

poses = np.concatenate([root_orient, body_pose, jaw_pose, eye_pose, hand_pose], axis=1)
mocap_frame_rate = 30
gender = 'male'
# trans = torch.zeros((batch_size, 3)).float().numpy()
trans = np.load('./temp.npy', allow_pickle=True)
betas = torch.zeros((10)).float().numpy()

paras = {
    'poses': poses,
    'gender': gender,
    'mocap_frame_rate': mocap_frame_rate,
    'trans': trans,
    'betas': betas
}

np.savez('test_smplx.npz', **paras)


dict_keys(['vertices', 'joints', 'global_orient', 'betas', 'body_pose'])
(196, 3)
(196, 10)
(196, 63)
hand_pose:(196, 90)
